# Segmentation evaluation after border thinning

Import libraries

In [1]:
import imagej
from skimage.io import imread_collection, imsave
import numpy as np
import os

Load FiJi plugins from local directory

In [2]:
ij = imagej.init('Fiji.app')

Added 387 JARs to the Java classpath.


Read Groundtruth and Results image files from directories

In [3]:
Ground_directory = './Evaluation/Groundtruth/*.png'
Ground_data = np.asarray(list(imread_collection(Ground_directory)),dtype='float32')
if (np.max(Ground_data)!=1):
    Ground_data=Ground_data/255.0
imsave('Ground.tif',Ground_data)
print('Groundtruth data shape is: ', np.shape(Ground_data))

Results_directory = './Evaluation/Results/*.png'
Results_data = np.asarray(list(imread_collection(Results_directory)), dtype='float32')
if (np.max(Results_data)!=1):
    Results_data=Results_data/255.0
imsave('Result.tif',Results_data)
print('Results data shape is: ', np.shape(Results_data))

Groundtruth data shape is:  (32, 512, 512)
Results data shape is:  (32, 512, 512)


Create ImageJ macros to run in BeanShell mode

In [4]:
Language_extension = "BeanShell"

macroVRand = """
import trainableSegmentation.metrics.*;
#@output String VRand
import ij.IJ;
originalLabels=IJ.openImage("C:/Users/akhadankish/Ground.tif");
proposedLabels=IJ.openImage("C:/Users/akhadankish/Result.tif");
metric = new RandError( originalLabels, proposedLabels );
maxThres = 1.0;
maxScore = metric.getMaximalVRandAfterThinning( 0.0, maxThres, 0.1, true );  
VRand = maxScore;
"""

macroVInfo = """
import trainableSegmentation.metrics.*;
#@output String VInfo
import ij.IJ;
originalLabels=IJ.openImage("C:/Users/akhadankish/Ground.tif");
proposedLabels=IJ.openImage("C:/Users/akhadankish/Result.tif");
metric = new VariationOfInformation( originalLabels, proposedLabels );
maxThres =1.0;
maxScore = metric.getMaximalVInfoAfterThinning( 0.0, maxThres, 0.1 );  
VInfo = maxScore;
"""

Calculate Foreground-restricted Rand Scoring (${V^{Rand}}_{thinned}$) and Foreground-restricted Information Theoretic Scoring (${V^{Info}}_{thinned}$) after border thinning

In [5]:
VRand = float(ij.py.run_script(Language_extension, macroVRand).getOutput('VRand'))
VInfo = float(ij.py.run_script(Language_extension, macroVInfo).getOutput('VInfo'))
print('Maximum foreground-restricted Rand score after thinning: ', VRand)
print('Maximum foreground-restricted information theoretic score after thinning: ', VInfo)

Maximum foreground-restricted Rand score after thinning:  0.9555808412810025
Maximum foreground-restricted information theoretic score after thinning:  0.9803861491590002
